# Determining Influencers of Netflix Stock
The goal of this project was to use identify "influencers" of Netflix stock and develop a method and general process for determining these people, issues, or topic influencers.  Due to the nature of Twitter data and the ability to capture real-time feedback, we used this platform as the primary data source for identifying influencers. The decision to use this data was warranted by the constraint to use publicly available data as Netflix private data would not be available to the team.  A primary assumption made in scoping the project is that all drivers of stock price changes can be captured through Twitter content.  These drivers include public opinion, economic and political conditions, and company earnings and performance information.  This assumption is based on the fact that Twitter is a social-media platform, widely used for the dissemination of news, public relations announcements, discussion of pop culture trends, individual opinion of news and entertainment, and for connecting socially.  Another assumption is that a two-year history of Twitter data would be sufficient for determining a method or algorithm for identifying influencers and is representative of all tweets concerning Netflix.  

Other sources of data included Netflix stock information, stock market indices, and Netflix financial news, all of which are publicly available datasets. 

### Data Collection
1. Two years of Twitter content information for the period beginning January 1, 2017 through February 28, 2019 was collected.  We were not able to use Twitter’s free API due to time period and message amount limitations.  Instead, we used a data scraping tool extension for Google Chrome (ScrapeHero, 2019) and tweets with keywords "Netflix" and "NFLX" during study period were scraped.  The Twitter data collected using this method is not as robust as the data that would have been obtained using Twitter’s API. Detailed retweet information, favorite information, follower information, and location coordinates were not provided using Twitter’s Advanced Search and the Google Chrome extension.

2. Python's Beautiful Soup was used to collect historical data on Netflix stock prices and stock indexes (Dow Jones, S&P 500 and Nasdaq) from the Yahoo Finance website (Yahoo Finance, 2019) and Quarterly EPS and financial reports, appointments and reliefs, as well as other industry related news from Netflix official investors website (Financial Releases, 2019). 


### Data Cleaning and Preprocessing

Twitter data was cleaned by remvoing the added web scraper app data, duplicate tweets, and tweets outside the time study period. Empty fields for favorites and retweets were filled with zeros and empty fields for missing names were filled with handles.  Non-English content was removed.    

### Feature Engineering

**Sentiment Scores**
To generate sentiment scores, URLs, images, hashtags, and mentions were separated from the Twitter content and the remaining content was converted to lowercase. With the aid of TextBlob and Vader sentiment (Hutto, C. J., & Gilbert, E, 2014) Python libraries, we generated 16 different sentiment scores for our database by either including or excluding punctuation, stop words, short words and lemmas (lemmatization).  We combined the best Vader and Polarity scores to generate a negative and a positive sentiment vector for the Twitter content. 

**Vectorization**
We used lowercased content, removed URLs, images, hashtags, mentions, and non-ascii characters; then n-grams of length 1 to 4 were generated, and with this data we built four sets of vectors. The first one used TF-IDF for the content words and counts for the mentions and hashtags together with the sentiment scores; the second set of vectors was similar to set one, except Twitter tokens were multiplied by the retweets column and sentiment scores were multiplied by the favorites column; for the third set of vectors, the content n-grams, hashtags and mentions were grouped and scored using TF-IDF; the sentiment was obtained by the mean of the TF-IDF multiplied by 0, 1, or -1, depending on the sentiment of tweets. This alternative was taken since the n-grams vectors were disjoint, even after using normalizations (Z-scores) and forcing the models to either ignore or acknowledge one set of attributes. Finally, the fourth set was the third vector set weighted by retweets and favorites in the same fashion as for the second set.  

**Feature Reduction**
L2 norm was approximated for each features and features of the order 10^(-10) were discarded and vectors with L2-norm equal to zero.

**Resampling**
Vectors were resampled to daily, weekly and monthly by adding frequencies (either counts or TF-IDF). From the official financial news, we manually created a vector for positive (1) and negative (-1) investor reaction to the financial report, based on the intra and inter day price change following the report. 

**Target Variable**
For supervised learning methods, we needed a target variable.  We used two different targets, one was the percentage price change of the weekly closing price, and the other was a one week shifted target to monitor the delay phenomena, under the assumption that a social trends impact might be delayed.

### Supervised Learning Methods

The goal was to use supervised learning for inference, rather than prediction, and to learn which features are associated with the target variable. We aimed to identify key features important to Netflix’s stock price.  Because of our goal to identify key features, regression models with easy interpretability were preferred and a few algorithms were shortlisted.  
 
The dataset used for the regression models was the set of 2,250 hashtag and mention vectors (raw weighted count), and the two positive and negative sentiment vectors.  The target variable was the intra-week stock price change percentage.  The initial dataset was split 80/20 into a train and test set using SciKit Learn’s train_test_split method.  The training and testing data was saved and the same sets were used for each different model. In addition, all models were validated and hyperparameters selected and tuned using cross fold validation. After final scoring of each model, no further hyperparameter tuning was performed.
 
The first of the models tested was the Random Forest Regressor. The Random Forest Regressor was used on several variations of the dataset, each described below.
 
Initially, the complete dataset of 2,250 features was fed into the Random Forest Regressor.  The model hyperparameters were chosen using the GridSearchCV method.  The hyperparameters tested were the number of estimators (trees in the forest), max features (number of features when looking for a split), and whether to use bootstrap sampling.  The root mean squared error (RMSE) was not great at .049, but since predictions were not the main focus of this project, more weight was placed on the R-squared (R2) value and the actual top features compared to what is known to have happened with Netflix over the last 2 years.
 
Two additional versions of this model were tested in an attempt to improve R2 and get a more meaningful top feature set.  The idea was to reduce noise and unimportant features prior to running the model through the Random Forest Regressor.  With that in mind, feature selection needed to happen first.  Two methods of feature selection were used: Recursive Feature Elimination (RFE) and forward-stepwise selection.  RFE was initialized to return the top 15 features.  This was a somewhat arbitrary decision, but we believed 15 features would be enough to gauge the quality of the model.  After running RFE and obtaining the top features, this reduced dataset was run through the Random Forest Regressor using a similar hyperparameter selection process as described above for the complete dataset.  The results of this model were an improved R2 of 0.424 but a slightly worse RMSE of 0.053. 
 
Finally, the last run of the Random Forest Regressor was with a dataset reduced using forward-stepwise selection.  A very basic form of stepwise selection was performed where the feature set was incremented one at a time and fit to an ordinary least squares model.  With each iteration, the t-test p-values for each feature were examined, and features with a p-value under 0.05 were eliminated.  This process resulted in a final set of 8 features, which can be seen in Table 2.  Using the same methods as the previous runs for hyperparameter tuning, this method resulted in an improved final R2 of 0.531 and a final RMSE of 0.05, which was comparable to the first Random Forest Regression run.

Two additional regression models were then run: Lasso and Ridge regression.  These models were used because they offered easy interpretability and the Sci-Kit Learn methods provide a coefficient attribute which would be used to determine top features.  In addition, Lasso and Ridge are known as “shrinkage” methods and help with feature selection by shrinking coefficient estimates.  The first run of Lasso returned very poor results, with only 1 feature having a coefficient above 0.  This model was dismissed because of this.  A first run of Ridge regression provided more promising results, so this model was selected as a final contender.  Hyperparameters were selected again using GridSearchCV.  The “best estimator” returned used an alpha, or regularization strength, of 5 and a Stochastic Average Gradient descent “saga” solver.  The final result of this model was an R2 of 0.446 and RMSE of 0.051.  When looking at the top 15 features using this model there were many similarities with the feature set returned by the RFE method and overall, this model provided meaningful results.  

### Unsupervised Learning Methods

We attempted to identify and learn about influencers from our large Twitter content database using clustering.

We implemented K-means, DBSCAN, Hierarchical, flat, spectral, and Kohonen Networks (Witten, I. H., Frank, E., Hall, M. A., & Pal, C. J. ,2016), to split our data into potential influencers or time periods to analyze. The criteria implemented for the number of clusters varied depending on the algorithm, for instance, in the case of the K-means, we followed the elbow criteria, and for the other clustering models, we used distances: Euclidean, Manhattan, Minkowski, in conjunction with cosine similarity, weighted cosine, and simple distances. For the particular case of Kohonen Nets, radial basis and L2 norm were used.  The clustering process generated a pool of 24 different ways to split our features, with an average of 80 clusters per algorithm. 

After the clusters were generated, we ran ordinary squares linear regressions for each of the clusters against the Netflix price percentage change within the same week and the one week shifted period hoping to assess the delay effect of the social media. The spirit was to identify the strong linear correlations, that is, R-squared greater than 0.5 with respect to both target and shifted target.  Furthermore, to select features within the cluster, avoid multicollinearity, and speed the learning process, we implemented principal component analysis (PCA) and under the Pareto rule 80-20, we kept only 80% of the variance from each cluster.  Additionally, within the same cluster there were similar elements; o eliminate similar elements, we ran scripts to group matching words with a probability of 0.85 to obtain the accumulated frequencies and top features. 

The above process yielded fifteen clusters with correlation coefficient greater than 0.5.  

### Resources

Financial Releases. (2019). Retrieved from https://www.netflixinvestor.com/investor-news-and-events/financial-releases/default.aspx

Hutto, C. J., & Gilbert, E. (2014, May). Vader: A parsimonious rule-based model for sentiment analysis of social media text. In Eighth international AAAI conference on weblogs and social media.

Netflix. (2019, April 24). Retrieved from https://en.m.wikipedia.org/wiki/Netflix

Pedregosa et al. (2011). Scikit-learn: Machine Learning in Python. Journal of Machine Learning Research, 2825-2830.

ScrapeHero. (2019, April 25). How to scrape Historical Twitter Data using Web Scraper and Chrome. Retrieved from https://www.scrapehero.com/how-to-scrape-historical-search-data-from-twitter/

Scikit, scikit-learn.org/stable/modules/classes.html#module-sklearn.cluster

Shaban, H. (2019, February 07). Twitter reveals its daily active user numbers for the first time. Retrieved from https://www.washingtonpost.com/technology/2019/02/07/twitter-reveals-its-daily-active-user-numbers-first-time/?noredirect=on&utm_term=.65ee1901c8dc

Statistics - (Shrinkage|Regularization) of Regression Coefficients. (n.d.). Retrieved from Nicome: https://gerardnico.com/data_mining/shrinkage

Twitter Advanced Search. (2019). Retrieved from https://twitter.com/search-advanced?lang=en

Yahoo Finance - Business Finance, Stock Market, Quotes, News. (2019). Retrieved from https://finance.yahoo.com/

Witten, I. H., Frank, E., Hall, M. A., & Pal, C. J. (2016). Data Mining: Practical machine learning tools and techniques. Morgan Kaufmann.

### Scripts and Data
**Financial_data.py**  To scrape indexes and Netflix data <br>
**Target_variable_shift.py** To compute target variable and shift it <br>
**Stock_resampling_momentum.py** . To identify oversold/overbought periods <br>
**Netflix_news.py**  To scrape Netflix’s official news <br>
**twitter.json**  To scrape Netflix tweets from Twitter <br>
**Twitter_DataAggregation.py**  To aggregate the twitter scraped data <br>
**Twitter_Database.py**  To pre-clean data, and remove the non-English <br>
**Twitter_Sentiment.py**  To clean and generated sentiment <br>
**Twitter_Sentiment_summary_results.py**  To validate sentiment (it generates metrics) <br>
**Twitter_vectors.py**  To clean and generate vectors from content using TF-IDF and counts for handles and mentions <br>
**Twitter_Vectors_tdidf.py**  To clean and generate vectors using TF-IDF for content, handles, and mentions <br>
**Twitter_clustering.py**  To generate all clusters <br>
**Twitter_clusters_similar.py**  To identify similar words within a cluster <br>
**netflix_Ridge.py**  Ridge Regression model <br>
**netflix_RandomRidgeRegressor.py**  Random Ridge Regressor model <br>
**netflix_RandomRidgeRegressor_RFE.py**  Random Ridge Regressor model using RFE feature selected set <br>
**netflix_RandomRidgeRegressor_stepwise.py** . Random Ridge Regressor model using stepwise feature selected set <br>
**netflix_regression_data_prep.py**  Dataframe, train and test set preparation for Supervised Learning scripts